In [11]:
# ================================================
# 1️⃣ Импорт библиотек
# ================================================
from typing import List, Tuple
import numpy as np
import re
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS
from datasets import load_dataset
from gensim.models import KeyedVectors
from gensim.models.phrases import Phrases, Phraser

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


# ================================================
# 2️⃣ Загрузка датасета
# ================================================
def load_sib200_ru():
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')

    X_train, y_train = trainset['text'], trainset['category']
    X_val, y_val = valset['text'], valset['category']
    X_test, y_test = testset['text'], testset['category']

    categories = sorted(list(set(y_train)))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

train_data, val_data, test_data, classes_list = load_sib200_ru()


# ================================================
# 3️⃣ Препроцессинг: чистая лемматизация
# ================================================
nlp = spacy.load("ru_core_news_sm", disable=["ner", "parser"])

def normalize_text(s: str, nlp_pipeline: spacy.Language):
    s = re.sub(r'https?://\S+|www\.\S+', '<URL>', s)
    s = re.sub(r'\S+@\S+', '<EMAIL>', s)
    s = re.sub(r'\d+', '<NUM>', s)
    doc = nlp_pipeline(s)
    tokens = []
    for token in doc:
        if token.is_punct or token.is_space or token.lower_ in STOP_WORDS:
            continue
        lemma = token.lemma_.lower()
        if len(lemma) <= 2:
            continue
        tokens.append(lemma)
    return tokens

print("🧹 Нормализация текста...")
X_train_norm = [normalize_text(it, nlp) for it in train_data[0]]
X_val_norm = [normalize_text(it, nlp) for it in val_data[0]]
X_test_norm = [normalize_text(it, nlp) for it in test_data[0]]


# ================================================
# 4️⃣ Биграммы / триграммы
# ================================================
print("🔗 Построение биграмм/триграмм...")
bigram = Phrases(X_train_norm, min_count=5, threshold=10, delimiter='_')
trigram = Phrases(bigram[X_train_norm], min_count=5, threshold=10, delimiter='_')
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

def make_ngrams(texts):
    return [trigram_mod[bigram_mod[toks]] for toks in texts]

X_train_norm = make_ngrams(X_train_norm)
X_val_norm = make_ngrams(X_val_norm)
X_test_norm = make_ngrams(X_test_norm)


# ================================================
# 5️⃣ TF-IDF + FastText
# ================================================
print("📈 Обучение TF-IDF...")
tfidf = TfidfVectorizer(
    tokenizer=lambda x: x,
    lowercase=False,
    sublinear_tf=True,
    min_df=3,
    max_df=0.8
)
tfidf.fit(X_train_norm)
idf_dict = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

print("💾 Загрузка FastText...")
MODEL_PATH_VEC = "./models/cc.ru.300.vec"
w2v = KeyedVectors.load_word2vec_format(MODEL_PATH_VEC, binary=False)

def sentence_vector_tfidf(tokens):
    vecs, weights = [], []
    for w in tokens:
        if w in w2v and w in idf_dict:
            vecs.append(w2v[w])
            weights.append(idf_dict[w])
    if not vecs:
        return np.zeros(w2v.vector_size)
    vecs = np.array(vecs)
    weights = np.array(weights).reshape(-1, 1)
    return np.sum(vecs * weights, axis=0) / np.sum(weights)


print("📐 Векторизация...")
X_train_vec = np.array([sentence_vector_tfidf(toks) for toks in X_train_norm])
X_val_vec = np.array([sentence_vector_tfidf(toks) for toks in X_val_norm])
X_test_vec = np.array([sentence_vector_tfidf(toks) for toks in X_test_norm])

# PCA + нормализация
pca = PCA(n_components=150, random_state=42)
X_train_vec = pca.fit_transform(X_train_vec)
X_val_vec = pca.transform(X_val_vec)
X_test_vec = pca.transform(X_test_vec)

X_train_vec = normalize(X_train_vec, norm='l2')
X_val_vec = normalize(X_val_vec, norm='l2')
X_test_vec = normalize(X_test_vec, norm='l2')


# ================================================
# 6️⃣ Logistic Regression (вместо SVM)
# ================================================
print("\n🚀 Обучаем Logistic Regression...")
pipeline = Pipeline([
    ('cls', LogisticRegression(max_iter=20000, class_weight='balanced', solver='liblinear'))
])

params = {
    'cls__C': [0.01, 0.05, 0.1, 0.5, 1],
    'cls__penalty': ['l2']
}

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train_vec, train_data[1])
best_model = grid.best_estimator_

print(f"\n✅ Лучшие параметры: {grid.best_params_}")
print(f"Лучший F1-macro (train CV): {grid.best_score_:.4f}")

print("\n📊 Валидация:")
y_val_pred = best_model.predict(X_val_vec)
print(classification_report(val_data[1], y_val_pred, target_names=classes_list))


# ================================================
# 7️⃣ Финальная модель и тест
# ================================================
# Объединяем train + va


🧹 Нормализация текста...
🔗 Построение биграмм/триграмм...
📈 Обучение TF-IDF...
💾 Загрузка FastText...


/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


📐 Векторизация...

🚀 Обучаем Logistic Regression...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END .......................cls__C=0.01, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.01, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.01, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.01, cls__penalty=l2; total time=   0.0s
[CV] END ........................cls__C=0.1, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.05, cls__penalty=l2; total time=   0.0s
[CV] END ........................cls__C=0.5, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.05, cls__penalty=l2; total time=   0.0s
[CV] END ........................cls__C=0.1, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.05, cls__penalty=l2; total time=   0.0s
[CV] END .......................cls__C=0.05, cls__penalty=l2; total time=   0.

/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either u